In [166]:
import pandas as pd
import os
import numpy as np
import csv
from sklearn.preprocessing import KBinsDiscretizer
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [64]:
current_dir = os.getcwd()

In [95]:
#dataset_path = os.path.join(current_dir, "../data/raw/dataset-kaggle.csv")

In [99]:
#df = pd.read_csv(dataset_path,header=None, index_col=None, names=[f"Item_{i}" for i in range(1, 21)])

In [18]:
# Exemplo de dataset
data = {
    'idade': [25, 30, 22, 28, 35],
    'categoria': ['A', 'B', 'A', 'A', 'C'],
    'experiencia': [5, 10, 3, 7, 15],
    'salario': [300, 700, 450, 200, 1000],
    'target': [1, 0, 1, 1, 0],  # Target 1 indica que o email é "spam"
}

# Criando o DataFrame
df = pd.DataFrame(data)

# Função para determinar o número de bins automaticamente
def calculate_bins(column):
    n = len(column)
    return int(np.ceil(np.log2(n) + 1))  # Regra de Sturges

# Aplicar discretização com n_bins dinâmico
num_cols = ['idade', 'experiencia', 'salario']
bin_counts = {col: calculate_bins(df[col]) for col in num_cols}

discretizer = KBinsDiscretizer(n_bins=[bin_counts[col] for col in num_cols], encode='ordinal', strategy='quantile')
df_discretized = pd.DataFrame(discretizer.fit_transform(df[num_cols]), 
                              columns=num_cols).astype(int)

# Mapear intervalos para nomes
for col, bins in zip(num_cols, discretizer.bin_edges_):
    df_discretized[col] = pd.cut(
        df[col], bins=bins, include_lowest=True, precision=1
    ).astype(str)

# Concatenar com dados categóricos
cat_cols = ['categoria']
df_transformed = pd.concat([df_discretized, df[cat_cols], df['target'], df['salario']], axis=1)

# Substituir valores NaN por uma string 'Unknown' (ou qualquer outro valor válido)
df_transformed.fillna('Unknown', inplace=True)

# Gerar condições como strings, incluindo o target
df_transformed['conditions'] = df_transformed.apply(
    lambda row: [f'{col}={val}' for col, val in row.items() if col not in ['target', 'salario']] + [f'target={row["target"]}'], axis=1
)

# Verificar as condições geradas
print("Condições geradas:")
print(df_transformed['conditions'])

# Criar DataFrame de transações para FP-Growth
transactions = pd.get_dummies(df_transformed['conditions'].explode()).groupby(level=0).max()

# Verificar as transações antes de aplicar FP-Growth
print("\nTransações geradas:")
print(transactions)

# Aplicar FP-Growth com um suporte menor (sem ponderação por enquanto)
frequent_itemsets = fpgrowth(transactions, min_support=0.1, use_colnames=True)

# Verificar se existem itemsets frequentes
if frequent_itemsets.empty:
    print("\nWarning: No frequent itemsets found. Consider lowering min_support further.")
else:
    print("\nItemsets frequentes encontrados:")
    print(frequent_itemsets)

# Gerar regras de associação
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

# Função para calcular o coverage das regras com o novo critério
def calculate_coverage(rule, target_col, transactions, df, max_coverage=0.05):
    # Identificar as transações que satisfazem a regra (antecedentes)
    antecedents = rule['antecedents']
    satisfied = transactions[antecedents].all(axis=1)
    
    # Calcular quantos exemplos do target = 1 são cobertos pela regra
    total_positive = sum(df[target_col] == 1)
    covered_positive = sum(satisfied & (df[target_col] == 1))
    
    # O coverage é a porcentagem dos positivos que a regra cobre
    coverage = covered_positive / total_positive if total_positive > 0 else 0
    
    # A regra será filtrada se o coverage for menor ou igual ao limite
    return coverage <= max_coverage

# Definir o coverage máximo desejado (5% do dataset com target = 1)
max_coverage = 0.05  # No máximo 5% do dataset com target = 1 (spam)

# Filtrar regras com coverage menor ou igual ao máximo
rules['coverage'] = rules.apply(lambda row: calculate_coverage(row, 'target', transactions, df_transformed, max_coverage), axis=1)
filtered_rules = rules[rules['coverage']]

# Mostrar as regras filtradas
print("\nFiltered Rules with Coverage <= 5%:")
print(filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'coverage']])


Condições geradas:
0    [idade=(21.9, 25.0], experiencia=(2.9, 5.0], c...
1    [idade=(28.0, 30.0], experiencia=(7.0, 10.0], ...
2    [idade=(21.9, 25.0], experiencia=(2.9, 5.0], c...
3    [idade=(25.0, 28.0], experiencia=(5.0, 7.0], c...
4    [idade=(30.0, 35.0], experiencia=(10.0, 15.0],...
Name: conditions, dtype: object

Transações geradas:
   categoria=A  categoria=B  categoria=C  experiencia=(10.0, 15.0]  \
0            1            0            0                         0   
1            0            1            0                         0   
2            1            0            0                         0   
3            1            0            0                         0   
4            0            0            1                         1   

   experiencia=(2.9, 5.0]  experiencia=(5.0, 7.0]  experiencia=(7.0, 10.0]  \
0                       1                       0                        0   
1                       0                       0                        1  

/Users/ieamedeiros/anaconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [19]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,coverage
0,(categoria=A),(target=1),0.6,0.6,0.6,1.0,1.666667,0.24,inf,1.000000,False
1,(target=1),(categoria=A),0.6,0.6,0.6,1.0,1.666667,0.24,inf,1.000000,False
2,"(idade=(21.9, 25.0])",(categoria=A),0.4,0.6,0.4,1.0,1.666667,0.16,inf,0.666667,False
3,"(idade=(21.9, 25.0])",(target=1),0.4,0.6,0.4,1.0,1.666667,0.16,inf,0.666667,False
4,"(categoria=A, idade=(21.9, 25.0])",(target=1),0.4,0.6,0.4,1.0,1.666667,0.16,inf,0.666667,False
...,...,...,...,...,...,...,...,...,...,...,...
149,"(experiencia=(10.0, 15.0], idade=(30.0, 35.0])","(target=0, categoria=C)",0.2,0.2,0.2,1.0,5.000000,0.16,inf,1.000000,True
150,"(categoria=C, idade=(30.0, 35.0])","(target=0, experiencia=(10.0, 15.0])",0.2,0.2,0.2,1.0,5.000000,0.16,inf,1.000000,True
151,"(experiencia=(10.0, 15.0])","(target=0, categoria=C, idade=(30.0, 35.0])",0.2,0.2,0.2,1.0,5.000000,0.16,inf,1.000000,True
152,(categoria=C),"(target=0, experiencia=(10.0, 15.0], idade=(30...",0.2,0.2,0.2,1.0,5.000000,0.16,inf,1.000000,True
